<a href="https://colab.research.google.com/github/shakin-shahria/Python_programming/blob/main/CSE_3812/Offline_2/Offline(2)_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Shakin Shahria
#011201055
import numpy as np

class Queen:
    def __init__(self, N):
        self.N = N
        self.queen_loc = dict()
        self.chess_board = [[0] * self.N for _ in range(self.N)]
        self.initialize = False

    # Tries to find the neighbor of the given cell.
    def get_neighbor(self, row, col):
        neighbor = []
        if 0 <= row - 1 < self.N and self.chess_board[row - 1][col] == 0:
            neighbor.append([row - 1, col])
        if 0 <= row + 1 < self.N and self.chess_board[row + 1][col] == 0:
            neighbor.append([row + 1, col])
        return neighbor

    # Print the current chessboard state and queen locations.
    def print_queen(self):
        print(self.chess_board)
        for Q in self.queen_loc:
            print(f'{Q}->{self.queen_loc[Q]}')

    # Check if two queens are attacking each other.
    def conflict(self, r1, c1, r2, c2):
        return (r1 == r2) or (c1 == c2) or (r1 + c1 == r2 + c2) or (r1 - c1 == r2 - c2)

    # Calculate the cost (number of conflicts) for a given state.
    def calc_cost(self, state):
        cost = 0
        max_cost = -999
        maxQ = None
        for Q in state:
            r1, c1 = state[Q]
            q_cost = sum(self.conflict(r1, c1, r2, c2) for q, (r2, c2) in state.items() if q != Q)
            cost += q_cost
            if q_cost > max_cost:
                max_cost = q_cost
                maxQ = Q
        return cost, max_cost, maxQ

    def goal_test(self, state):
        return self.calc_cost(state)[0] == 0

    # Get all possible neighboring states by moving one queen at a time.
    def get_all_neighbors(self, state):
        all_neighbors = []
        for Q, (row, col) in state.items():
            neighbors = self.get_neighbor(row, col)
            for neighbor in neighbors:
                neighbor_state = state.copy()
                neighbor_state[Q] = neighbor
                all_neighbors.append(neighbor_state)

        return all_neighbors

    # Update the chessboard with the current queen positions.
    def update_chess_board(self):
        self.chess_board = [[0] * self.N for _ in range(self.N)]
        for Q, (row, col) in self.queen_loc.items():
            self.chess_board[row][col] = Q

    # Adds new queen to the chessboard and updates the chess_board
    def add_queen(self, row, col):
        self.queen_loc[f"Q{len(self.queen_loc)}"] = [row, col]
        self.update_chess_board()


In [ ]:
def steepest_ascent_hill_climbing(self, max_iterations=1000):
    current_state = self.queen_loc.copy()
    current_cost, _, _ = self.calc_cost(current_state)

    iteration_states = []  # List to store states for each iteration
    iteration_costs = []  # List to store costs for each iteration

    for i in range(max_iterations):
        iteration_states.append(current_state.copy())
        iteration_costs.append(current_cost)

        if current_cost == 0:  # Solution found
            break

        all_neighbors = self.get_all_neighbors(current_state)
        best_neighbor_state = None
        best_neighbor_cost = current_cost

        for neighbor_state in all_neighbors:
            neighbor_cost, _, _ = self.calc_cost(neighbor_state)
            if neighbor_cost < best_neighbor_cost:
                best_neighbor_state = neighbor_state
                best_neighbor_cost = neighbor_cost

        if best_neighbor_cost < current_cost:
            current_state = best_neighbor_state
            current_cost = best_neighbor_cost
        else:
            # Check if the best neighbor state is the same as the current state
            if best_neighbor_state == current_state:
                break  # No better neighbor found, terminate the search

    self.queen_loc = current_state
    self.update_chess_board()

    # Append the last state and cost after the search ends
    iteration_states.append(current_state.copy())
    iteration_costs.append(current_cost)

    return iteration_states, iteration_costs

# Assign the steepest_ascent_hill_climbing method to the Queen class
Queen.steepest_ascent_hill_climbing = steepest_ascent_hill_climbing

# Create a Queen object with a board size of 4
queen = Queen(4)

# Manually set the initial positions of the queens
queen.add_queen(2, 0)
queen.add_queen(1, 1)
queen.add_queen(2, 2)
queen.add_queen(1, 3)

queen.initialize = True  # Set to True after manually setting the initial positions

print("Initial Chessboard:")
queen.print_queen()

# Run steepest ascent hill climbing and get the iteration states and costs
iteration_states, iteration_costs = queen.steepest_ascent_hill_climbing()

print("\nFinal Chessboard (After Steepest Ascent Hill Climbing):")
queen.print_queen()

# Print each iteration's state and cost
print("\nIterations:")
for i, state in enumerate(iteration_states):
    print(f"Iteration {i + 1}:")
    queen.queen_loc = state
    queen.update_chess_board()
    queen.print_queen()
    print(f"Heuristic Value (Cost): {iteration_costs[i]}")
    print("-" * 20)


In [ ]:
def simulated_annealing(self, initial_temperature=100.0, cooling_rate=0.01, max_iterations=1000):
    current_state = self.queen_loc.copy()
    current_cost, _, _ = self.calc_cost(current_state)

    iteration_states = []  # List to store states for each iteration
    iteration_costs = []  # List to store costs for each iteration

    temperature = initial_temperature

    for i in range(max_iterations):
        iteration_states.append(current_state.copy())
        iteration_costs.append(current_cost)

        if current_cost == 0:  # Solution found
            break

        all_neighbors = self.get_all_neighbors(current_state)
        random_neighbor_state = np.random.choice(all_neighbors)  # Choose a random neighbor state

        neighbor_cost, _, _ = self.calc_cost(random_neighbor_state)
        delta_cost = neighbor_cost - current_cost

        # Decide whether to move to the random neighbor or not
        if delta_cost < 0 or np.random.rand() < np.exp(-delta_cost / temperature):
            current_state = random_neighbor_state
            current_cost = neighbor_cost

        # Update the temperature for the next iteration
        temperature *= (1.0 - cooling_rate)

    self.queen_loc = current_state
    self.update_chess_board()  # Corrected function name

    # Append the last state and cost after the search ends
    iteration_states.append(current_state.copy())
    iteration_costs.append(current_cost)

    return iteration_states, iteration_costs


# Attach the simulated_annealing method to the Queen class
Queen.simulated_annealing = simulated_annealing

# Create a Queen object with a board size of 4
queen = Queen(4)

# Manually set the initial positions of the queens
queen.add_queen(2, 0)
queen.add_queen(1, 1)
queen.add_queen(2, 2)
queen.add_queen(1, 3)

queen.initialize = True  # Set to True after manually setting the initial positions

print("Initial Chessboard:")
queen.print_queen()

# Run simulated annealing and get the iteration states and costs
iteration_states, iteration_costs = queen.simulated_annealing()

print("\nFinal Chessboard (After Simulated Annealing):")
queen.print_queen()

# Print each iteration's state and cost
print("\nIterations:")
for i, state in enumerate(iteration_states):
    print(f"Iteration {i + 1}:")
    queen.queen_loc = state
    queen.update_chess_board()
    queen.print_queen()
    print(f"Heuristic Value (Cost): {iteration_costs[i]}")
    print("-" * 20)
